# Landweber iteration and discrepancy principle
In this lab, you should solve a three-dimensional tomography problem with the Landweber method. Have a look in the course literature to see how this method works. Here are some general hints:

* Start by choosing an appropriate discretized space with three dimensions and symmetric min/max points around zero, starting with $32 \times 32 \times 32$ resolution for testing, and initialize it with odl. *If you do not have access to a suitable GPU, or if you have not installed the optional dependency with astra-toolbox, then this lab must be done in two dimensions.*

* Create a three-dimensional [Shepp–Logan phantom](https://odlgroup.github.io/odl/generated/odl.phantom.transmission.shepp_logan.html) in the space. Use ``show`` to make sure that the image are centered at the origin. 

* Form the [ray transform operator](http://odlgroup.github.io/odl/generated/odl.tomo.operators.ray_trafo.RayTransform.html) using a [cone beam geometry](http://odlgroup.github.io/odl/generated/odl.tomo.geometry.conebeam.cone_beam_geometry.html#odl.tomo.geometry.conebeam.cone_beam_geometry) with a small number $(\sim 10)$ of angles, otherwise you will not see the effects of semi-convergence. 

* Apply the operator to the phantom to obtain the sinogram, and add some noise to the sinogram (which lives in the space ``Operator.range``). 

* You get the norm of a space element with its [norm method](https://odlgroup.github.io/odl/generated/odl.discr.discr_space.DiscretizedSpaceElement.norm.html#odl.discr.discr_space.DiscretizedSpaceElement.norm). For the discrepancy in the Landweber method, you will need the noise level $\| g - g_{\mathrm{noisy}}\|$ of the sinogram $g$. (In real applications you must estimate the noise level, but let us keep this example simple.)

* Get the adjoint of the forward operator, e.g., ``Operator.adjoint``.

* You can obtain an estimate of the largest singular value of the forward operator with its [norm method](http://odlgroup.github.io/odl/generated/odl.operator.operator.Operator.norm.html). (Note that this is the operator norm, which may need to be estimated with ``estimate=True`` if it is not directly available/precomputed).

* Implement Landweber and employ $\sim 200$ iterations to obtain a sequence of reconstructions $f_n$ from the noisy sinogram, starting with the [zero vector](https://odlgroup.github.io/odl/generated/odl.discr.discr_space.DiscretizedSpace.zero.html#odl.discr.discr_space.DiscretizedSpace.zero) of the space.

* Log the relevant norms between iterations and check if and when the discrepancy criterion is satisfied, that is, when $\| Operator(f_n) - g_{\mathrm{noisy}} \| \approx \|g - g_{\mathrm{noisy}}\|$ (but do not stop the iterations).

* Plot the relative error of your reconstructions against the number of iterations, and see what the discrepancy criterion would give you.

* Compare your solution to that obtained with the [built-in version of Landweber](https://odlgroup.github.io/odl/generated/odl.solvers.iterative.iterative.landweber.html#odl.solvers.iterative.iterative.landweber).

In [ ]:
import odl
import numpy as np
import matplotlib.pyplot as plt